# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import ollama

In [2]:
# Initialize and constants


# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = "llama3.2"

In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            self.body = response.content
        except Exception as e:
            print(f"Failed to fetch {url}: {e}")
            self.body = ""
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edward

In [9]:
def get_links(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        format="json",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ]
    )
    result = response['message']['content']
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/MiniMaxAI/MiniMax-M2',
 '/deepseek-ai/DeepSeek-OCR',
 '/moonshotai/Kimi-Linear-48B-A3B-Instruct',
 '/meituan-longcat/LongCat-Video',
 '/briaai/FIBO',
 '/models',
 '/spaces/HuggingFaceTB/smol-training-playbook',
 '/spaces/enzostvs/deepsite',
 '/spaces/khang119966/DeepSeek-OCR-DEMO',
 '/spaces/Wan-AI/Wan2.2-Animate',
 '/spaces/merterbak/DeepSeek-OCR-Demo',
 '/spaces',
 '/datasets/nvidia/PhysicalAI-Autonomous-Vehicles',
 '/datasets/HuggingFaceFW/finewiki',
 '/datasets/neulab/agent-data-collection',
 '/datasets/nvidia/Nemotron-VLM-Dataset-v2',
 '/datasets/AlicanKiraz0/Turkish-SFT-Dataset-v1.0',
 '/datasets',
 '/join',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/inference/models',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Int

In [11]:
get_links("https://huggingface.co/")

ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [36]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    for link in links["links"]:
        print("Trying:", link["url"])
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [37]:
print(get_all_details("https://huggingface.co/"))

Trying: https://huggingface.co
Trying: https://huggingface.co/models
Trying: https://huggingface.co/datasets
Trying: https://huggingface.co/spaces
Trying: https://huggingface.co/about
Trying: https://huggingface.co/join
Trying: https://huggingface.co/enterprise
Trying: https://huggingface.co/pricing
Trying: https://huggingface.co/changelog
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
deepseek-ai/DeepSeek-OCR
Updated
about 12 hours ago
•
302k
•
1.57k
PaddlePaddle/PaddleOCR-VL
Updated
2 days ago
•
11.8k
•
996
tencent/HunyuanWorld-Mirror
Updated
1 day ago
•
2.41k
•
272
Qwen/Qwen3-VL-8B-Instruct
Updated
8 days ago
•
179k
•
303
nanonets/Nanonets-OCR2-

In [50]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [51]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [52]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Trying: https://huggingface.co/
Trying: https://huggingface.co/brand
Trying: https://apply.workable.com/huggingface/
Trying: https://blog.huggingface.co/
Failed to fetch https://blog.huggingface.co/: HTTPSConnectionPool(host='blog.huggingface.co', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000023A4E687E60>: Failed to resolve 'blog.huggingface.co' ([Errno 11001] getaddrinfo failed)"))
Trying: https://discuss.huggingface.co/
Trying: https://status.huggingface.co/
Trying: https://github.com/huggingface
Trying: https://twitter.com/huggingface


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-OCR\nUpdated\nabout 12 hours ago\n•\n302k\n•\n1.57k\nPaddlePaddle/PaddleOCR-VL\nUpdated\n2 days ago\n•\n11.8k\n•\n997\ntencent/HunyuanWorld-Mirror\nUpdated\n1 day ago\n•\n2.41k\n•\n274\nQwen/Qwen3-VL-8B-Instruct\nUpdated\n8 days ago\n•\n179k\n•\n304\nnanonets/Nanonets-OCR2-3B\nUpdated\n7 days ago\n•\n19.9k\n•\n391\nBrowse 1M+ models\nSpaces\nRunning\n490\n490\nveo3.1-f

In [53]:
def create_brochure(company_name, url):
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response["message"]["content"]
    display(Markdown(result))

In [45]:
create_brochure("HuggingFace", "https://huggingface.co/")

Trying: https://huggingface.co/
Trying: https://endpoints.huggingface.co
Trying: https://blog.huggingface.co
Failed to fetch https://blog.huggingface.co: HTTPSConnectionPool(host='blog.huggingface.co', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000023A4EBB1F10>: Failed to resolve 'blog.huggingface.co' ([Errno 11001] getaddrinfo failed)"))
Trying: https://discuss.huggingface.co
Trying: https://status.huggingface.co
Failed to fetch https://status.huggingface.co: HTTPSConnectionPool(host='status.huggingface.co', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000023A4E6FC380>: Failed to resolve 'status.huggingface.co' ([Errno 11002] getaddrinfo failed)"))
Trying: https://github.com/huggingface
Trying: https://twitter.com/huggingface
Trying: https://www.linkedin.com/company/huggingface/


**Hugging Face: The AI Community Building the Future**
=====================================================

**About Us**
-----------

Hugging Face is a collaborative platform for the machine learning community to build, discover, and collaborate on AI models, datasets, and applications. Our mission is to accelerate innovation in AI research and deployment.

**Our Culture**
--------------

At Hugging Face, we value:

*   **Community**: We believe that collaboration and open sharing of knowledge are key to driving progress in AI.
*   **Innovation**: We strive to develop cutting-edge tools and models that make it easier for researchers and practitioners to build and deploy AI applications.
*   **Openness**: We foster an environment where our users can contribute, learn from each other, and share their work with the world.

**Our Customers**
----------------

We serve a diverse range of customers, including:

*   **Researchers**: Academics and scientists working on cutting-edge AI research projects.
*   **Practitioners**: Industry professionals building AI applications for various domains, such as computer vision, natural language processing, and more.
*   **Developers**: Individuals and companies developing AI-powered products and services.

**Our Products**
----------------

Hugging Face offers a comprehensive suite of tools and models for building, deploying, and managing AI applications. Our flagship features include:

*   **Transformers**: A state-of-the-art library for transformer-based models.
*   **Diffusers**: A cutting-edge library for diffusion-based models.
*   **Tokenizers**: Fast and efficient tokenizers optimized for research and production.

**Our Career Opportunities**
-----------------------------

Join our team of passionate individuals working on the forefront of AI research and development. We offer a range of career paths, including:

*   **Software Engineering**: Help us develop and maintain our flagship products.
*   **Research**: Collaborate with our community to advance the state-of-the-art in AI.
*   **Sales and Support**: Work with our customers to understand their needs and provide exceptional support.

**Our Impact**
-------------

Hugging Face is proud to have:

*   **Supported over 50,000 organizations** in building and deploying AI applications.
*   **Fostered a community of over 4.2 million followers**, including researchers, practitioners, and developers.
*   **Published numerous research papers** on the latest advancements in AI.

Join us today and be part of the Hugging Face community!

[Sign up](https://huggingface.co/) | [Learn more](https://about.huggingface.co/)

### Output by the Humourous Prompt

In [54]:
create_brochure("HuggingFace", "https://huggingface.co/")

Trying: https://huggingface.co/
Trying: https://huggingface.co/brand
Trying: https://apply.workable.com/huggingface/


# Hugging Face: Building a Future in AI

[Cover Image: A group of people from diverse backgrounds collaborating on a computer, with a cityscape in the background]

Welcome to Hugging Face, where machine learning enthusiasts come together to build a better future. Our platform is more than just a collection of models and datasets – it's a community that's passionate about accelerating AI research and applications.

## Our Story

We started as a simple idea: provide a platform for machine learning researchers to share their work and collaborate with others. Today, we're proud to be one of the largest open-source platforms in the industry, with over 1 million models and counting.

## What We Offer

* **Models**: Explore our vast collection of pre-trained models, or build your own using our easy-to-use APIs.
* **Datasets**: Access a growing library of datasets, from text to image to audio files.
* **Spaces**: Create and share applications, from chatbots to video generators, with just a few clicks.
* **Community**: Join our vibrant community of developers, researchers, and enthusiasts who are shaping the future of AI.

## Our Impact

More than 50,000 organizations trust Hugging Face for their AI needs, including companies like Meta, Amazon, Google, Intel, Microsoft, and Grammarly. Our platform has been used in a wide range of applications, from text analysis to computer vision.

## Join the Movement

Want to be part of something exciting? Explore our [Models](#models), [Datasets](#datasets), and [Spaces](#spaces) today!

### Models
 Browse 1M+ models, or build your own using our easy-to-use APIs. [Learn More](#models)

### Datasets
Access a growing library of datasets, from text to image to audio files. [Explore Now](#datasets)

### Spaces
Create and share applications with just a few clicks. [Get Started](#spaces)

### Community
Join our vibrant community of developers, researchers, and enthusiasts who are shaping the future of AI. [Meet Us](#community)

---

[Call-to-Action: Sign up for free today and start building your own AI projects]

Note: This is just a sample brochure, you can add more details and images as per your requirement.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>